In [ ]:
import os

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import seaborn as sns
import io
from contextlib import redirect_stdout
from matplotlib.colors import ListedColormap
import matplotlib as mpl

from clean_plots import *

sns.set_style("whitegrid")

OUT = io.StringIO()

In [ ]:
OFFSETS = 0.01 * (np.arange(11) - 5)
OFFSET_COLORS = sns.color_palette("BrBG", 11)
OFFSET_MAP = dict(zip(OFFSETS, OFFSET_COLORS))
OFFSET_CMAP = ListedColormap(OFFSET_COLORS)
sns.palplot(OFFSET_COLORS)

In [ ]:
def plot_distribution(offset=OFFSETS):
    count_energy = {oi: getallspinconfig(offset=oi) for oi in offset}
    count = {
        oi: {
            spin_energy[0]: count_energy[oi][spin_energy]
            for spin_energy in count_energy[oi]
        }
        for oi in count_energy
    }
    energy = {
        oi: {spin_energy[0]: spin_energy[1] for spin_energy in count_energy[oi]}
        for oi in count_energy
    }
    # Xlabel = [tuple([int(q) for q in list(format(i, '05b'))]) for i in range(2 ** 5)]
    # X = np.arange(2 ** 5)
    Xlabel = [(0, 1, 0, 0, 0), (1, 0, 0, 1, 0), (1, 1, 1, 1, 1)]
    X = np.array(range(len(Xlabel)))

    fig, ax = plt.subplots(figsize=p["figsize"])
    for idx, os in enumerate(offset):
        """set X"""
        xdraw = X - 0.5 + idx / (len(offset) + 1) + 0.5 / (len(offset) + 1)
        width = 1 / (len(offset) + 1)
        """plot dwave result
        """
        height = []
        for Xi in Xlabel:
            if Xi in count[os]:
                height.append(count[os][Xi])
            else:
                height.append(0)
        if os < 0:
            color = green
        elif os > 0:
            color = yellow
        else:
            color = blue
        ax.bar(
            x=xdraw,
            height=height,
            width=width,
            color=OFFSET_COLORS[idx],
            label="D-Wave 2000Q" if idx == 0 else None,
            zorder=1,
            alpha=0.9,
            align="edge",
        )
        """plot simulation result
        """

        with redirect_stdout(OUT):
            prob = getallspintheory(os)
        height = [prob[Xi] for Xi in Xlabel]
        if idx == len(offset) - 1:
            label = "Simulation"
        else:
            label = None
        ax.bar(
            x=xdraw,
            height=height,
            width=width,
            linewidth=1,
            color="none",
            edgecolor="k",
            align="edge",
            label=label,
            zorder=2,
        )

    """labels
    """
    ax.set_xlabel("Final State Distribution", p["textargs"])
    ax.set_ylabel("Ground State Probability", p["textargs"])
    ax.set_xticks(X)
    ax.set_xticklabels(Xlabel, rotation="0")
    ax.legend(loc="best", fancybox=True, shadow=False, frameon=True)

    cax = fig.add_axes([0.27, -0.07, 0.5, 0.05])
    cbar = mpl.colorbar.ColorbarBase(
        cax,
        cmap=OFFSET_CMAP,
        norm=mpl.colors.Normalize(vmin=-0.05, vmax=0.05),
        orientation="horizontal",
        label="Offset",
    )

    return fig

In [ ]:
def plot_annealcurve(offsets=[0.00, 0.05]):
    fig, ax = plt.subplots(figsize=p["figsize"])
    for os in offsets:
        label = f"{os:0.2f}"

        if False:  # os==0.0:
            colorA = colorB = "k"
            ls = "--"
        else:
            colorA = colorB = OFFSET_MAP[os]
            ls = "-"
        alpha = 1
        with redirect_stdout(OUT):
            s, yA, yB = getannealcurve(os, normalized_time=[-0.1, 1.1])
        ax.errorbar(x=s, y=yA, color=colorA, marker="None", ls=ls, label=label)
        ax.errorbar(x=s, y=yB, color=colorB, alpha=alpha, marker="None", ls=ls)
    """labels
       """
    ax.set_xlabel("Annealing time $s$ ($\mu s$)", p["textargs"])
    ax.set_ylabel("$A_i(s)$ and $B_i(s)$ (GHz)", p["textargs"])
    ax.set_xlim(0, 1)

    ax.legend(loc="right", title="Offset")

    return fig

In [ ]:
def plot_spectrum():
    fig, ax = plt.subplots(figsize=p["figsize"])
    ax.set_xlabel("Annealing Time $s$ ($\mu s$)", **p["textargs"])
    ax.set_ylabel("Dimensionless Energy", **p["textargs"])
    with redirect_stdout(OUT):
        timegrid, r, gap, gap_density = getspectrum(0.0)
    colors = sns.color_palette("deep", 11)
    ax.set_xlim(0, 1)
    ax.set_ylim(-0.5, 3.5)

    for idx, es in enumerate(r):
        ax.errorbar(x=timegrid, y=r[es], marker="None", ls="--", color=colors[idx])
        fig.savefig(
            f"spectrum_states_{idx}.pdf", transparent=False, bbox_inches="tight"
        )
    return fig

In [ ]:
def plot_all():
    fig, ax = plt.subplots(figsize=p["figsize"], nrows=1, sharex=True)
    """dwave result
    """
    offset = np.array(list(0.01 * (np.arange(11) - 5)))[::-1]
    graphsize = list(np.arange(2, 13))
    with redirect_stdout(OUT):
        prob = {os: [getall(os, gs) for gs in graphsize] for os in offset}

    # ax = axs[0]
    for os in offset:
        color = OFFSET_MAP[os]
        ax.errorbar(
            x=graphsize,
            y=prob[os],
            ls="-",
            marker="o",
            color=color,
            alpha=os_alpha[os],
            label="D-Wave 2000Q" if os == 0.05 else None,
        )
    degeneracy = np.array([degenfcn(Xi) for Xi in graphsize])
    rguess = np.array([1 / 2 ** xi for xi in graphsize]) * degeneracy
    ax.errorbar(
        x=graphsize, y=rguess, ls="--", marker="s", color="k", label="Random Guessing"
    )
    ax.set_ylabel("Ground State Probability", p["textargs"])

    ax.set_xlabel("Graph Size $v$", p["textargs"])
    plt.legend()
    ax.set_yscale("log")
    ax.set_ylim(1.0e-3)

    cax = fig.add_axes([0.95, 0.27, 0.025, 0.5])
    cbar = mpl.colorbar.ColorbarBase(
        cax,
        cmap=OFFSET_CMAP,
        norm=mpl.colors.Normalize(vmin=-0.05, vmax=0.05),
        label="Offset",
    )

    return fig, ax

In [ ]:
def plot_offset0():
    fig, axs = plt.subplots(
        figsize=(p["figsize"][0], p["figsize"][1] * 2), nrows=2, sharex=True
    )
    """dwave result
    """
    graphsize = list(np.arange(2, 13))
    with redirect_stdout(OUT):
        prob = [getall(0, gs) for gs in graphsize]

    ax = axs[0]
    color = "k"
    ax.errorbar(
        x=graphsize,
        y=prob,
        ls="-",
        marker="o",
        color=blue,
        alpha=1,
        label="D-Wave 2000Q",
    )
    degeneracy = np.array([degenfcn(Xi) for Xi in graphsize])
    rguess = np.array([1 / 2 ** xi for xi in graphsize]) * degeneracy
    ax.errorbar(
        x=graphsize, y=rguess, ls="--", marker="s", color="k", label="Random Guessing"
    )
    ax.set_ylabel("Ground State Probability", p["textargs"])
    ax.set_yscale("log")
    ax.legend()

    ax = axs[1]
    color = "k"
    ax.errorbar(
        x=graphsize,
        y=prob / rguess,
        ls="-",
        marker="o",
        color=blue,
        alpha=1,
        label="D-Wave 2000Q",
    )
    degeneracy = np.array([degenfcn(Xi) for Xi in graphsize])
    rguess = np.array([1 / 2 ** xi for xi in graphsize]) * degeneracy
    ax.set_ylabel("Probability Ratio: D-Wave/Random", p["textargs"])
    ax.set_xlabel("Graph Size $v$", p["textargs"])
    plt.tight_layout()

    # ax.set_ylim(1.e-3)

    return fig, ax

In [ ]:
def plot_tdse():
    fig, ax = plt.subplots(figsize=p["figsize"])
    """dwave result
    """
    # offset = list(0.01 * (np.arange(15) - 7))
    offset = list(0.01 * (np.arange(11) - 5))
    prob = []
    with redirect_stdout(OUT):
        for os in offset:
            try:
                prob.append(gettdse(os))
            except:
                prob.append(np.nan)

    print(prob)

    ax.errorbar(x=offset, y=prob, ls="--", marker="o", color=blue, label="D-Wave 2000Q")
    """simulation result
    """
    offset = list(0.01 * (np.arange(11) - 5))
    prob = []
    with redirect_stdout(OUT):
        for os in offset:
            try:
                prob.append(gettdsetheory(os, normalized_time=[0, 1]))
            except:
                prob.append(np.nan)
    ax.errorbar(x=offset, y=prob, ls="--", marker="s", color="k", label="Simulaton")
    """labels
    """
    ax.set_xlabel("Offset", p["textargs"])
    ax.set_ylabel("Ground State Probability", p["textargs"])
    ax.legend()

    print(prob)

    return fig